In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


Autosaving every 180 seconds


In [2]:
#############################################
######### LOAD SUITE2P AND BINARIZE #########
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210301/'
# data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/'
# data_dir = '/media/cat/4TB/donato/nathalie/plane0'
# data_dir = '/media/cat/4TB/donato/renan/renan'
data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228'

# initialize calcium object and load suite2p data
c = calcium.Calcium()
c.verbose = True             # outputs additional information during processing
c.recompute = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
c.data_dir = data_dir
c.load_suite2p()             # this function assumes output dirs is data_dir/

# set flags to save matlab and python data
c.save_python = True         # save output as .npz file 
c.save_matlab = True         # save output as .mat file

###############################################
##### PARAMETERS FOR RUNNING BINARIZATION #####
###############################################
c.min_width_event_onphase = c.sample_rate//2 # set minimum withd of an onphase event; default: 0.5 seconds
c.min_width_event_upphase = c.sample_rate//4 # set minimum width of upphase event; default: 0.25 seconds

############# PARAMTERS TO TWEAK ##############
#     1. Cutoff for calling somthing a spike:
#        This is stored in: std_Fluorescence_onphase/uppohase: defaults: 1.5
#                                        higher -> less events; lower -> more events
#                                        start at default and increase if data is very noisy and getting too many noise-events
c.min_thresh_std_onphase = 2.5      # set the minimum thrshold for onphase detection; defatul 2.5
c.min_thresh_std_upphase = 2.5      # set the minimum thershold for uppohase detection; default: 2.5

#     2. Filter of [Ca] data which smooths the data significantly more and decreases number of binarzied events within a multi-second [Ca] event
#        This is stored in high_cutoff: default 1.0
#                             - the lower we set it the smoother our [Ca] traces and less "choppy" the binarized traces
c.high_cutoff = 0.5              

#     3. Removing bleaching and drift artifacts using polynomial fits
#        This is stored in detrend_model_order
c.detrend_model_order = 5 # 1-5 polynomial fit


################################################
########### RUN BINARIZATION STEP ##############
################################################
# 
c.binarize_fluorescence()



WARNING ***** Found cells with 0-[Ca] traces :  179
  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (1027, 92900)
         self.Fneu (neuropile):  (1027, 92900)
         self.iscell (Suite2p cell classifier output):  (1788, 2)
         # of good cells:  (1027,)
         self.spks (deconnvoved spikes):  (1027, 92900)
         self.stat (footprints structure):  (1027,)
         mean std over all cells :  23.5

  Binarization parameters: 
        low pass filter low cuttoff:  0.5 hz
        oasis_thresh_prefilter:  15
        min_thresh_std_oasis:  0.1
        min_thresh_std_onphase:  1.5
        min_thresh_std_upphase:  1.5
        min_width_event_onphase:  15
        min_width_event_upphase:  7
        min_width_event_oasis:  2
        min_event_amplitude:  1


model filter: remove bleaching or trends: 100%|████████████████| 1027/1027 [00:23<00:00, 42.84it/s]


WARNING ***** Found cells with 0-[Ca] traces :  179


binarizing continuous traces filtered fluorescence onphase: 100%|█| 1027/1027 [00:19<00:00, 51.47it
binarizing continuous traces filtered fluorescence upphase: 100%|█| 1027/1027 [00:38<00:00, 26.72it


   Oasis based binarization skipped by default ... 


In [3]:
###############################################################
######### LOAD BINARZIED DATA AND VISUALIZE CELL ACTIVITY #####
###############################################################
#fname = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/nathalie/plane0/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/renan/renan/binarized_traces.npz'
# 
fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/binarized_traces.npz'

c = calcium.Calcium()
c.fname = fname
c.load_binarization()

# pick a random cell to visuzlie
cell_id = 307
scale = 100

# 
c.plot_cell_binarization(cell_id, scale)


(1027, 92900)


In [6]:
######################################################
############# SHOW RASTERS ###########################
######################################################
fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/binarized_traces.npz'

c = calcium.Calcium()
c.fname = fname
c.load_binarization()

c.show_rasters()


(1027, 92900)
